In [2]:
import gspread
import os
from dotenv import load_dotenv
import pandas as pd

# json 파일이 위치한 경로를 값으로 줘야 합니다.

json_file_path = os.environ.get('GCP_JSON_PATH')

gc = gspread.service_account(json_file_path)

spreadsheet_url = os.environ.get('SHEET_URL')
doc = gc.open_by_url(spreadsheet_url)

main_worksheet = doc.worksheet("총명단")
python_attendance = doc.worksheet("Python 문법 기초반")

In [6]:
df = pd.read_csv("zoom_logs/8기 파문기_1주차.csv")
jun_time = df[df['게스트'] == '아니요']['기간(분)'][0]
df['이름(원래 이름)'] = df['이름(원래 이름)'].apply(lambda x : x.split('(')[0].replace(' ','').replace('_',''))
range_list = [x.value for x in python_attendance.range('A4:A400')]
merge_df = pd.merge(pd.DataFrame(range_list, columns=['이름(원래 이름)']), df.groupby(['이름(원래 이름)'])['기간(분)'].sum().reset_index(), on='이름(원래 이름)', how='left').fillna(0)
merge_df
merge_df['출첵'] = merge_df['기간(분)'].apply(lambda x : 'O' if jun_time-10 <= x else ('지각' if x != 0 else 'X'))
results_df = merge_df[merge_df['이름(원래 이름)'] != '']
results_df

,이름(원래 이름),기간(분),출첵
0,강빈21,12.0,지각
1,강서윤69,0.0,X
2,강수빈08,58.0,O
3,강예은46,59.0,O
4,강지웅11,57.0,O
...,...,...,...
220,한소리67,56.0,O
221,한혜인88,0.0,X
222,황정현60,54.0,O
223,황지희73,0.0,X


In [8]:
cell_list = python_attendance.range(f'F4:F{4+len(results_df)}')

for i, val in enumerate(results_df['출첵']):  #gives us a tuple of an index and value
    cell_list[i].value = val    #use the index on cell_list and the val from cell_values

python_attendance.update_cells(cell_list)

{'spreadsheetId': '1pmYiv4pl_ec3sE7puBIuyuqaMy2nlCiPIvhfA_oRoP8',
 'updatedRange': "'Python 문법 기초반'!F4:F229",
 'updatedRows': 226,
 'updatedColumns': 1,
 'updatedCells': 226}